Файл повторяет парсинг позиций в точности до $\pm$10 строк.

In [1]:
import pandas as pd
import requests as rq
import numpy as np
from datetime import datetime, timedelta
import os

# Демонстрация того, как будет происходить процесс совмещения на двух днях

In [2]:
# Ссылка на данные
url1 = "https://iss.moex.com/iss/history/engines/stock/markets/shares/sessions/2/securities/LKOH.json?from=2023-11-07&till=2023-11-07&table_type=full"
url2 = "https://iss.moex.com/iss/history/engines/stock/markets/shares/sessions/2/securities/LKOH.json?from=2023-11-08&till=2023-11-08&table_type=full"

In [3]:
# Отправляем GET-запрос по указанной ссылке
response = rq.get(url1)

# Проверяем статус ответа
if response.status_code == 200:
    # Загружаем данные из ответа в переменную data в случае успеха
    data = response.json()
else:
    print("Ошибка при получении данных")
    

columns = response.json()["history"]['columns']

data = response.json()["history"]["data"]

df1 = pd.DataFrame(data, columns=columns)


# Отправляем GET-запрос по указанной ссылке
response = rq.get(url2)

# Проверяем статус ответа
if response.status_code == 200:
    # Загружаем данные из ответа в переменную data в случае успеха
    data = response.json()
else:
    print("Ошибка при получении данных")


columns = response.json()["history"]['columns']

data = response.json()["history"]["data"]

df2 = pd.DataFrame(data, columns=columns)

In [4]:
print(df1.sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1))

    TRADEDATE SECID                   BOARDNAME BOARDID SHORTNAME  \
0  2023-11-07  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   

      REGNUMBER          ISIN LISTNAME  FACEVALUE CURRENCYID  ...  \
0  1-01-00077-A  RU0009024277        1      0.025        SUR  ...   

  MARKETPRICE3  MARKETPRICE3TRADESVALUE DECIMALS          TYPE  \
0         None                        0        1  common_share   

   CLOSEAUCTIONPRICE  WAVAL  LASTPRICE  MARKETPRICE3TRADESVALUECUR  \
0                  0      0     7333.5                        None   

  MARKETPRICE3CUR  TRADINGSESSION  
0            None               2  

[1 rows x 53 columns]


In [5]:
frames = [df1, df2]

result = pd.concat(frames).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1)
filename = "aboba_date.csv"
result.to_csv(filename, sep=',', index=False, encoding='utf-8')

print(result)

    TRADEDATE SECID                   BOARDNAME BOARDID SHORTNAME  \
0  2023-11-07  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
1  2023-11-08  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   

      REGNUMBER          ISIN LISTNAME  FACEVALUE CURRENCYID  ...  \
0  1-01-00077-A  RU0009024277        1      0.025        SUR  ...   
1  1-01-00077-A  RU0009024277        1      0.025        SUR  ...   

  MARKETPRICE3  MARKETPRICE3TRADESVALUE DECIMALS          TYPE  \
0         None                        0        1  common_share   
1         None                        0        1  common_share   

   CLOSEAUCTIONPRICE  WAVAL  LASTPRICE  MARKETPRICE3TRADESVALUECUR  \
0                  0      0     7333.5                        None   
1                  0      0     7295.0                        None   

  MARKETPRICE3CUR  TRADINGSESSION  
0            None               2  
1            None               2  

[2 rows x 53 columns]


Далее будет пробегать по массиву дат и таким образом сформируем всю таблицу

Ниже я задаю массив всех дат

# Парсинг данных данных

Сначала мы создаём массив, который содержить все даты, за которые мы хотим получить данные

In [6]:
start_date = datetime(2020, 5, 8)
end_date = datetime.now()

current_date = start_date
array_date = []
while current_date <= end_date:
    array_date += [current_date.strftime("%Y-%m-%d")]
    # print(current_date.strftime("%Y-%m-%d"))
    current_date += timedelta(days=1)
# print(array_date[-13])
# print((datetime.strptime(array_date[-13], '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d"))

Тут мы формируем массив из дней, а затем, если сервер работает хорошо, то возвращаем данные за весь промежуток от 5 августа 2020 до сегодняшнего дня. 

Если сервер перестаёт отвечать, тогда полученные данные мы собираем в файл и при повторном запуске программа проверить наши файлы и начнёт с последнего записанного дня.

В качестве отладки печатается дата дня, который мы сейчас получаем. 

In [7]:
def add_new_day(date_today: str, ticker_name: str):
    date_yesterday = (datetime.strptime(date_today, '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d")
    url = f"https://iss.moex.com/iss/history/engines/stock/markets/shares/sessions/2/securities/{ticker_name}.json?from={date_today}&till={date_today}&table_type=full"

    while True:
        try:
            # Ваш код, который может вызвать ошибку
            response = rq.get(url)
            # break  # Если ошибки не возникло, выходим из цикла
        except Exception as e:
            # Обработка других исключений
            print(f"Произошла ошибка: {e}")
        else:
            break

    # Проверяем статус ответа
    if response.status_code != 200:
        print("Ошибка при получении данных")
        add_new_day(date_today)

    df_for_add = pd.DataFrame(response.json()["history"]["data"], columns=response.json()["history"]['columns'])
    df_for_add.sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1)
   

    return df_for_add

# ticker_names=['si', 'br', 'ri', 'eu', 'mx', 'sr', 'ox', 'gz', 'yn', 'fv', 'vi', 'lk']

ticker_names=['LKOH', "FIVE", "YNDX", "GAZP", "SBER"]

for ticker_name in ticker_names:
    if os.path.exists(f"{ticker_name}_full_date_price.csv"):
        df = pd.read_csv(f"{ticker_name}_full_date_price.csv")
        last_valid_date_index = df['TRADEDATE'].last_valid_index()
        last_valid_date_value  = df['TRADEDATE'][last_valid_date_index]
        date_tomorrow = (datetime.strptime(last_valid_date_value, '%Y-%m-%d') + timedelta(days=1)).strftime("%Y-%m-%d")
        print(date_tomorrow)
    # Вывод первых нескольких строк DataFrame для проверки
        # print(not date_tomorrow in array_date)
        if(not date_tomorrow in array_date):
            pass
        else:
            df_array = [df]
            for date in array_date[array_date.index(date_tomorrow)+1:]:
                #  print(df)
                # df = add_new_day(date, ticker_name)
                df_array += [add_new_day(date, ticker_name)]
                # print(df)
                print(date)
            df = pd.concat(df_array).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1)
            print(df)
            filename = f"{ticker_name}_full_date_price.csv"
            df.to_csv(filename, sep=',', index=False, encoding='utf-8')
    else:
        print(f"Файл '{ticker_name}_full_date_price.csv' не существует.")
        df_array = []
        for date in array_date:
            df_array += [add_new_day(date, ticker_name)]
            print(date)
        df = pd.concat(df_array).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1)
        print(df)
        filename = f"{ticker_name}_full_date_price.csv"
        df.to_csv(filename, sep=',', index=False, encoding='utf-8')

    

Файл 'LKOH_full_date_price.csv' не существует.
2020-05-08
2020-05-09
2020-05-10
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-16
2020-05-17
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-23
2020-05-24
2020-05-25
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-05-30
2020-05-31
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-06
2020-06-07
2020-06-08
2020-06-09
2020-06-10
2020-06-11
2020-06-12
2020-06-13
2020-06-14
2020-06-15
2020-06-16
2020-06-17
2020-06-18
2020-06-19
2020-06-20
2020-06-21
2020-06-22
2020-06-23
2020-06-24
2020-06-25
2020-06-26
2020-06-27
2020-06-28
2020-06-29
2020-06-30
2020-07-01
2020-07-02
2020-07-03
2020-07-04
2020-07-05
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-12
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-18
2020-07-19
2020-07-20
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-25
2020-07-26
2020-07-27
2020-07-28
2020-07-29
2020-07-30
2020-07-31
2020-08-01
2020-08

C:\Users\rromm\AppData\Local\Temp\ipykernel_8580\850257227.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_array).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1)


      TRADEDATE SECID                   BOARDNAME BOARDID SHORTNAME  \
0    2020-06-22  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
1    2020-06-23  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
2    2020-06-25  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
3    2020-06-26  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
4    2020-06-29  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
..          ...   ...                         ...     ...       ...   
870  2023-11-24  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
871  2023-11-27  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
872  2023-11-28  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
873  2023-11-29  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   
874  2023-11-30  LKOH  Т+: Акции и ДР - безадрес.    TQBR    ЛУКОЙЛ   

        REGNUMBER          ISIN LISTNAME  FACEVALUE CURRENCYID  ...  \
0    1-01-00077-A  RU0009024277        1      0.025        SUR  ...   
1    

C:\Users\rromm\AppData\Local\Temp\ipykernel_8580\850257227.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_array).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1)


      TRADEDATE SECID                   BOARDNAME BOARDID SHORTNAME REGNUMBER  \
0    2020-08-03  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   
1    2020-08-04  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   
2    2020-08-05  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   
3    2020-08-06  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   
4    2020-08-07  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   
..          ...   ...                         ...     ...       ...       ...   
842  2023-11-24  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   
843  2023-11-27  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   
844  2023-11-28  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   
845  2023-11-29  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   
846  2023-11-30  FIVE  Т+: Акции и ДР - безадрес.    TQBR  FIVE-гдр      None   

             ISIN LISTNAME 

...

      TRADEDATE SECID                   BOARDNAME BOARDID   SHORTNAME  \
0    2020-06-22  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   
1    2020-06-23  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   
2    2020-06-25  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   
3    2020-06-26  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   
4    2020-06-29  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   
..          ...   ...                         ...     ...         ...   
870  2023-11-24  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   
871  2023-11-27  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   
872  2023-11-28  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   
873  2023-11-29  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   
874  2023-11-30  YNDX  Т+: Акции и ДР - безадрес.    TQBR  Yandex clA   

    REGNUMBER          ISIN LISTNAME  FACEVALUE CURRENCYID  ... MARKETPRICE3  \
0        None  NL0009805522        1       

C:\Users\rromm\AppData\Local\Temp\ipykernel_8580\850257227.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_array).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1)


       TRADEDATE SECID                              BOARDNAME BOARDID  \
0     2020-06-22  GAZP  Т+: Неполные лоты (акции) - безадрес.    SMAL   
1     2020-06-22  GAZP             Т+: Акции и ДР - безадрес.    TQBR   
2     2020-06-23  GAZP  Т+: Неполные лоты (акции) - безадрес.    SMAL   
3     2020-06-23  GAZP             Т+: Акции и ДР - безадрес.    TQBR   
4     2020-06-25  GAZP  Т+: Неполные лоты (акции) - безадрес.    SMAL   
...          ...   ...                                    ...     ...   
1575  2023-11-28  GAZP             Т+: Акции и ДР - безадрес.    TQBR   
1576  2023-11-29  GAZP  Т+: Неполные лоты (акции) - безадрес.    SMAL   
1577  2023-11-29  GAZP             Т+: Акции и ДР - безадрес.    TQBR   
1578  2023-11-30  GAZP  Т+: Неполные лоты (акции) - безадрес.    SMAL   
1579  2023-11-30  GAZP             Т+: Акции и ДР - безадрес.    TQBR   

       SHORTNAME     REGNUMBER          ISIN LISTNAME FACEVALUE CURRENCYID  \
0     ГАЗПРОМ ао  1-02-00028-A  RU0007661625 

C:\Users\rromm\AppData\Local\Temp\ipykernel_8580\850257227.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(df_array).sort_values(by=['TRADEDATE']).reset_index().drop('index', axis=1)


       TRADEDATE SECID                              BOARDNAME BOARDID  \
0     2020-06-22  SBER             Т+: Акции и ДР - безадрес.    TQBR   
1     2020-06-23  SBER             Т+: Акции и ДР - безадрес.    TQBR   
2     2020-06-25  SBER  Т+: Неполные лоты (акции) - безадрес.    SMAL   
3     2020-06-25  SBER             Т+: Акции и ДР - безадрес.    TQBR   
4     2020-06-26  SBER             Т+: Акции и ДР - безадрес.    TQBR   
...          ...   ...                                    ...     ...   
1503  2023-11-28  SBER             Т+: Акции и ДР - безадрес.    TQBR   
1504  2023-11-29  SBER  Т+: Неполные лоты (акции) - безадрес.    SMAL   
1505  2023-11-29  SBER             Т+: Акции и ДР - безадрес.    TQBR   
1506  2023-11-30  SBER  Т+: Неполные лоты (акции) - безадрес.    SMAL   
1507  2023-11-30  SBER             Т+: Акции и ДР - безадрес.    TQBR   

     SHORTNAME  REGNUMBER          ISIN LISTNAME FACEVALUE CURRENCYID  ...  \
0     Сбербанк  10301481B  RU0009029540      

Удаление повторяющихся данных (в силу того, как МосБиржа возвращала данные (в выходные дни вместо пустого запроса она возвращает последнюю цену за последний рабочий день, то есть 14*20 одинаковых записей, в силу чего эта ячейка позволяла сэкономить, примерно, 300 МБ))

In [8]:
for ticker_name in ticker_names:
    your_file = f"{ticker_name}_full_date_price.csv" 
# Загрузка данных
    df = pd.read_csv(your_file)

# Удаление повторяющихся строк
    unique_rows = df.drop_duplicates()

# Сохранение результата
    unique_rows.to_csv(your_file, index=False)

Итоговые данные не идеальны, поэтому мы их будем обрабатывать в других данных.